# Phase 1 Notebook: `policy.py`

Target file: `execution/langgraph/policy.py`

Purpose: define deterministic memoization policy decisions and memo key generation for heavy deterministic tasks.

## 3-Layer Context

- Layer 1 (Directive): Phase 1 requires reliability controls and memory behavior.
- Layer 2 (Orchestration): `graph.py` asks this policy whether memoization is required.
- Layer 3 (Execution): policy is deterministic, side-effect free, and unit-testable.

In [ ]:
from pathlib import Path
import sys

def bootstrap_repo_root() -> Path:
    cwd = Path.cwd().resolve()
    candidates = [cwd, *cwd.parents, Path('/home/nir/dev/agent_phase0')]
    for candidate in candidates:
        if (candidate / 'execution' / 'langgraph' / 'policy.py').exists():
            if str(candidate) not in sys.path:
                sys.path.insert(0, str(candidate))
            return candidate
    raise RuntimeError('Could not locate repo root for policy notebook.')

repo_root = bootstrap_repo_root()
print('repo_root =', repo_root)
print('kernel_python =', sys.executable)
if '/.venv/' not in sys.executable.replace('\\', '/'):
    print('WARNING: kernel is not the project .venv interpreter.')

## P0 vs P1 (Memoization Decisioning)

| Concern | Phase 0 | Phase 1 (`policy.py`) | Why this matters |
|---|---|---|---|
| Memo requirement logic | Not explicit | `requires_memoization(...)` | Repeatable behavior, fewer LLM-only decisions |
| Key strategy | Ad hoc path strings | `suggested_memo_key(...)` | Predictable lookup/reuse in tooling and tests |
| Retry boundaries | Generic loop retries | explicit `max_policy_retries` | bounded failure for skipped memoization |

## Why this block exists: inspect policy methods

We inspect the policy class to verify heavy-write detection and deterministic key derivation are encoded in one place.

In [ ]:
import inspect
from execution.langgraph.policy import MemoizationPolicy

print(inspect.getsource(MemoizationPolicy))

## Why this block exists: validate heavy vs light behavior

Policy must trigger on expensive deterministic writes (for example Fibonacci dumps) and not on trivial writes.

In [ ]:
policy = MemoizationPolicy(max_policy_retries=2)

heavy_args = {'path': 'fib.txt', 'content': ','.join(str(i) for i in range(120))}
light_args = {'path': 'note.txt', 'content': 'ok'}
heavy_result = {'result': 'Successfully wrote 900 characters to fib.txt'}
light_result = {'result': 'Successfully wrote 2 characters to note.txt'}

heavy_required = policy.requires_memoization(tool_name='write_file', args=heavy_args, result=heavy_result)
light_required = policy.requires_memoization(tool_name='write_file', args=light_args, result=light_result)
heavy_required, light_required

In [ ]:
assert heavy_required is True
assert light_required is False
key = policy.suggested_memo_key(tool_name='write_file', args={'path': 'fib.txt'}, result={'result': 'ok'})
assert key == 'write_file:fib.txt'
print('policy assertions passed')

## Takeaways

- Policy file turns memoization from prompt preference into deterministic governance.
- P1 introduces bounded retry logic via explicit policy signals.
- This file is pure logic, so it should stay fast and test-first.
- Next notebook: `execution/notebooks/p1_memo_store.ipynb`.